### Consulta 4
###### def best_developer_year( año : int ): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [12]:
import pandas as pd
import re

Guardamos en sus repectectivas variables las rutas de acceso

In [13]:
df_games = pd.read_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\user_games_clean.parquet')
df_reviews = pd.read_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\user_reviews_clean.parquet')
df_sentimientos = pd.read_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Sentiment_analysis\Sentiment_Analysis_parquet')


Creamos una copia de cada DataFrame para realizar sobre ellas las operaciones 

In [14]:
df_games_copy = df_games.copy()
df_reviews_copy = df_reviews.copy()
df_sentimientos_copy = df_sentimientos.copy()

Fusionamos df1 y df2 en item_id utilizando un left join (manteniendo todas las filas de df1)

In [15]:
df_merged = df_sentimientos_copy.merge(df_games_copy[['id', 'developer']], left_on='item_id', right_on='id', how='left')

#### El código realiza las siguientes acciones:

1. Define un patrón regex para extraer el año (cuatro dígitos).
2. Extrae el año de cada celda en la columna 'posted' utilizando regex y lo guarda en una nueva columna llamada 'year_posted'.
3. Elimina la columna original 'posted' del DataFrame.


In [16]:

patron = r'(\b\d{4}\b)'

df_merged['year_posted'] = df_merged['posted'].str.extract(patron)

df_merged.drop(columns=['posted'], inplace=True)



 Reordenamos las columnas del DataFrame con el nuevo orden


In [17]:
nuevo_orden_columnas = [
    'user_id',
    'item_id',
    'recommend',
    'sentiment_analysis',
    'developer',
    'year_posted'
]

# Reordena las columnas del DataFrame con el nuevo orden
df_merged = df_merged[nuevo_orden_columnas]

Eliminamos las filas con valores NaN en la columna 'year_posted'



In [18]:
df_merged.dropna(subset=['year_posted'], inplace=True)


Convierte la columna 'year_posted' al tipo de datos entero


In [19]:
df_merged['year_posted'] = df_merged['year_posted'].astype(int)


Creamos la consulta, donde devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [20]:
def best_developer_year(df, año):
    """
    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado.
    Se tienen en cuenta recommend = True + sentiment_analysis con mayor puntuación.

    Args:
        df (pd.DataFrame): DataFrame que contiene los datos.
        año (int): Año para el que se desea obtener el top 3 de desarrolladores.

    Returns:
        dict: Diccionario ordenado con los top 3 desarrolladores para el año dado, en formato {1er puesto: primer juego, 2do puesto: segundo juego, 3er puesto: tercer juego}.
    """
    # Filtrar el DataFrame para el año dado
    df_año = df[df['year_posted'] == año]

    # Filtrar por recomendaciones verdaderas y sentimiento de análisis más alto
    df_filtrado = df_año[df_año['recommend'] & (df_año['sentiment_analysis'] == df_año['sentiment_analysis'].max())]

    # Agrupar por desarrollador y contar las recomendaciones
    top_developers = df_filtrado.groupby('developer')['recommend'].sum()

    # Ordenar los desarrolladores por número de recomendaciones en orden descendente
    top_developers = top_developers.sort_values(ascending=False)

    # Tomar los top 3 desarrolladores
    top_3_developers = top_developers.head(3)

    # Crear el diccionario ordenado con los top 3 desarrolladores
    top_developers_dict = {}
    for i, (developer, recomendaciones) in enumerate(top_3_developers.items(), 1):
        puesto = f"{i}er puesto"
        top_developers_dict[puesto] = developer

    return top_developers_dict

Realizamos la prueba de su correcto funcionamiento

In [21]:
# Llamar a la función con el año deseado
año = 2015
top_3_desarrolladores = best_developer_year(df_merged, año)

# Imprimir el top 3 de desarrolladores para el año dado
print(f"Top 3 de desarrolladores para el año {año}: {top_3_desarrolladores}")


Top 3 de desarrolladores para el año 2015: {'1er puesto': 'Valve', '2er puesto': 'Facepunch Studios', '3er puesto': 'Smartly Dressed Games'}


Exportamos la consulta a .parquet

In [22]:
df_merged.to_parquet(r'C:\Users\Usuario\Desktop\proyecto steam\Data Exportada\df_marged.parquet', index=False)